# Options Pricing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.interpolate as interp
import scipy.stats as st

%matplotlib inline

## Options

An option is a financial derivatives that give buyers the right, *but not the obligation*, to buy or sell an underlying asset at an agreed-upon price and date.

An option is defined by:

* A strike price, $K$: This is the price at which an individual is allowed to buy/sell the asset
* Expiry date, $T$: The amount of time until the option expires and can no longer be exercised

A *call option* is an agreement in which the purchaser of the option is allowed to purchase the stock at the strike price

A *put option* is an agreement in which the purchaser of the option is allowed to sell the stock at the strike price

**Disclaimer**: The approach taken in this lecture is simplified but more complex versions of what we explore are used by financial professionals.



**Two "styles" of options**

* European: European options can only be exercised at the expiration date $T$.
* American: American options allow one to exercise the option at any moment up to, and including, $T$.


How much will a call option be worth if the stock price is above the strike price?

It is worth the difference between the expiry stock price and the strike price because one could exercised and then sold at market value.

How much will a call option be worth if the stock price is below the strike price?

It is worth 0 at expiry because no one would choose to exercise at that price.

## Pricing an option

Let $S_t$ denote the price of the stock at period $t$. We assume that the stock price follows,

\begin{align*}
  \log \frac{S_{t+1}}{S_{t}} &\sim N(0, \sigma)
\end{align*}

i.e.

\begin{align*}
  \log S_{t+1} &= \log S_t + \sigma \varepsilon \\
  \sigma &\sim N(0, 1)
\end{align*}

There is a risk-free rate of $r$ which provides an "outside option" and is used to discount future cash flows -- i.e. receiving $S_T - K$ in period is only worth $\left( \frac{1}{1+r} \right)^T (S_T - K)$

In [ ]:
# Parameters
log_S0 = 0.0  # S0 = 1
sigma = 0.015
T = 24  # time period of 2 years
K = 1.05  # Strike price of 1.05

# Annualized interest rate of 2.5%
r = (1 + 0.025)**(1/12) - 1

In [ ]:
def simulate_stock_price(log_S0, sigma, T):
    # Can generate a rw by using cumulative sum of random draws
    log_St = log_S0 + np.cumsum(sigma*np.random.randn(T))

    return np.exp(log_St)

In [ ]:
fig, ax = plt.subplots()

for i in range(1_500):
    ax.plot(simulate_stock_price(log_S0, sigma, T), alpha=0.025, color="g")

# zorder ensures that the black dotted line is on top
ax.hlines(K, 0, T, color="k", linestyle="--", zorder=1_000_000)

## Pricing a European option

A European call option will only be exercised if $S_T > K$ thus the value of a European option is given by

\begin{align*}
  V(S_0) &= \left( \frac{1}{1+r} \right)^T E \left[\max\{0, S_T - K\}\right] \\
  &= \left( \frac{1}{1+r} \right)^T \left((\text{Prob}(S_T < K) 0 + \text{Prob}(S_T \geq K) E[ S_T - K | S_T > K] \right)
\end{align*}

### Pricing it numerically

We can simulate the outcome of prices which means that we can compute the equation above

In [ ]:
S_T_numerical = np.array(
    [simulate_stock_price(log_S0, sigma, T)[-1] for n in range(10_000)]
)

european_price_numerical = np.mean((1 / (1 + r)**T) * np.maximum(0, S_T_numerical - K))
print(european_price_numerical)

### A little knowledge of log-normal random variables...

We may have turned to numerical methods more quickly than needed -- This particular framework allows us to price things analytically:

Note that we can express the price in period $T$ as

\begin{align*}
  \log S_T &= \log S_0 + \sum \sigma \varepsilon \\
  &= \log S_0 + \sigma \sqrt{T} \tilde{\varepsilon}
\end{align*}


We are interested in

\begin{align*}
  V_0 &= \left( \frac{1}{1+r} \right)^T \left((\text{Prob}(S_T < K) 0 + \text{Prob}(S_T \geq K) E[ S_T - K | S_T > K] \right) \\
  &= \left( \frac{1}{1+r} \right)^T \text{Prob}(S_T \geq K) E[ S_T - K | S_T > K] \\
  &= \left( \frac{1}{1+r} \right)^T (1 - \text{Prob}(S_T < K)) E[ S_T - K | S_T > K] \\
  &= \left( \frac{1}{1+r} \right)^T (1 - \text{Prob}(S_T < K)) \exp\left\{\log S_0 + \frac{\sigma^2 T}{2} \right\} \frac{\Phi\left[\frac{\log S_0 + \sigma^2 T - \log(K)}{\sigma \sqrt{T}}\right]}{1 - \Phi\left[ \frac{\log K - \log S_0}{\sigma \sqrt{T}} \right]}\\
\end{align*}

See the section on [Conditional Expectations of log-normal random variables](https://en.wikipedia.org/wiki/Log-normal_distribution#Conditional_expectation) for the derivation

In [ ]:
d_ST = st.lognorm(scale=np.exp(log_S0), s=sigma*np.sqrt(T))
d_sn = st.norm()
mu = log_S0
sig = sigma*np.sqrt(T)

p1 = 1 / (1 + r)**T
p2 = 1 - d_ST.cdf(K)
p3 = np.exp(mu + sig**2/2)
p3n = d_sn.cdf((mu + sig**2 - np.log(K)) / sig)
p3d = 1 - d_sn.cdf((np.log(K) - mu) / sig)

european_price_analytical = p1 * p2 * (p3 * p3n/p3d - K)

**Comparing numerical and analytical solutions**

In [ ]:
print("Analytical price is: ", european_price_analytical)
print("Numerical price is: ", european_price_numerical)

## Pricing an American option

An American option can be exercised at any point in time prior to expiration.

In the worse case, one could treat an American option as if it were a European option, but one also gets the benefit of being able to exercise prior to expiration. This means that an equally specified American option should cost *no less* than a European option.

Would we be willing to pay more?

### Dynamic programming to price an American option

Let $V_t(S_t)$ denote the value of owning an American option with strike price $K$, time to expiration $T - t$, and current stock price $S_t$.

How would we express this value mathematically?

- Until after we exercise the option, we always can choose to exercise in the current period and collect $S_t - K$.
- Until we exercise the option and prior to expiration, we can choose to wait and see what the price is tomorrow.
- At expiry, it has the same value as the European option -- $\max \{0, S_t - K\}$

This type of problem is known as an "optimal stopping problem". We must choose a point at which to "stop" and find out more. Other problems that fit into this framework:

* Capital acquisition for factories
* Portfolio balancing
* Search theory
* [A problem that stumped Milton Friedman](https://python.quantecon.org/wald_friedman.html)

**Bellman equation**

All dynamic programs should have a Bellman equation! This will look slightly different than our infinite horizon cake-eating problem because there's a fixed amount of time in which the option can be exercised.

\begin{align*}
  V_t(S_t) &= \max_{\text{N}, \text{Y}} \{0 + \frac{1}{1 + r} E[V_{t+1}(S_{t+1})], S_t - K\}
\end{align*}

**Solution via value function iteration**

We solve this Bellman equation using a form value function iteration!

This is slightly different because we're not necessarily looking for a fixed point -- We instead have a terminal condition, $V_T(S_T) = \max\{0, S_T - K\}$, and we are just iterating backwards to find $V_0$.

In [ ]:
class CallOptionPricer(object):
    """
    Helps compute European and American call option prices
    """
    def __init__(self, log_S0, sigma, K, T, r):
        self.log_S0 = log_S0
        self.S0 = np.exp(log_S0)
        self.sigma = sigma
        self.K = K
        self.T = T
        self.r = r

    def european_price(self):
        "Computes the European call option price"
        # Unpack useful variables
        log_S0, S0 = self.log_S0, self.S0
        sigma = self.sigma
        K, T, r = self.K, self.T, self.r

        # Build useful random variables
        _mu = log_S0
        _sig = sigma*np.sqrt(T)
        d_ST = st.lognorm(scale=S0, s=_sig)
        d_sn = st.norm()

        p1 = 1 / (1 + r)**T
        p2 = 1 - d_ST.cdf(K)
        p3 = np.exp(_mu + _sig**2/2)
        p3n = d_sn.cdf((_mu + _sig**2 - np.log(K)) / _sig)
        p3d = 1 - d_sn.cdf((np.log(K) - _mu) / _sig)

        return p1 * p2 * (p3 * p3n/p3d - K)

    def american_price(self, nS=51):
        "Computes the American call option price"
        # Unpack useful variables
        log_S0, S0 = self.log_S0, self.S0
        sigma = self.sigma
        K, T, r = self.K, self.T, self.r
        _mu = log_S0
        _sig = sigma*np.sqrt(T)

        # Build a grid that we'll iterate on
        logSgrid = np.linspace(log_S0-2.5*_sig, log_S0+2.5*_sig, nS)
        a_t = np.zeros((T, nS))
        V_t = np.zeros((T, nS))

        # Compute the terminal condition
        V_t[-1, :] = np.maximum(0, np.exp(logSgrid) - K)

        # Iterate over time periods
        for t in range(T-1, -1, -1):
            V_tp1 = interp.interp1d(
                logSgrid, V_t[t, :],
                fill_value="extrapolate"
            )

            # Iterate over the states
            for (i, log_St) in enumerate(logSgrid):
                S_t = np.exp(log_St)
                log_Stp1 = log_St + sigma*np.random.randn(50_000)
                EV_tp1 = np.mean(V_tp1(log_Stp1))

                # Value of exercising
                V_exercise = S_t - K
                V_defer = 0 + (1 / (1+r)) * EV_tp1

                a_t[t-1, i] = 0 if V_defer >= V_exercise else 1
                V_t[t-1, i] = np.maximum(V_exercise, V_defer)

        # Compute the American price V_0(S_0)
        V_0 = interp.interp1d(logSgrid, V_t[0, :])
        american_price = V_0(log_S0)*1

        return a_t, V_t, logSgrid, american_price


**Results**

In [ ]:
cop = CallOptionPricer(log_S0, sigma, K, T, r)

In [ ]:
print("The European price is: ", cop.european_price())

In [ ]:
a_t, V_t, logSgrid, american_price = cop.american_price(nS=151)
print("The American price is: ", american_price)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Make data.
Svals = np.exp(logSgrid)
Tvals = np.arange(0, T)
SS, TT = np.meshgrid(Svals, Tvals)

VV = np.zeros_like(SS)
for t in Tvals:
    V_interp = interp.interp1d(logSgrid, V_t[t, :])
    VV[t, :] = V_interp(logSgrid)

# Plot the surface.
surf = ax.plot_surface(
    SS, TT, VV, cmap="viridis",
    linewidth=0, antialiased=False,
)

ax.view_init(elev=10., azim=240)

ax.set_xlabel("S_t")
ax.set_ylabel("t")
ax.set_title("American Option Prices")

In [ ]:
fig, ax = plt.subplots()

ax.pcolormesh(SS, TT, a_t, shading="nearest")